<a href="https://colab.research.google.com/github/LakshmiP1/DS/blob/main/multi_linear_toyota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.regressionplots import influence_plot
import statsmodels.formula.api as smf
import numpy as np

In [ ]:
#Load the data set
cars = pd.read_csv("Toyoto_Corrola.csv")
cars.head()

In [ ]:
cars.info()

In [ ]:
# dropping the case number columns as it is not required
cars.drop(["Id","Model"],inplace=True,axis = 1)

In [ ]:
cars.info()

### **Correlation** **Matrix**

In [ ]:
cars.corr()

In [ ]:
cars.Cylinders.describe()

In [ ]:
cars.drop('Cylinders',inplace=True,axis=1)


In [ ]:
cars.info()

In [ ]:
cars.corr()

# **Scatter Plot b/w variables and along with histogram**

In [ ]:
#Format the plot background and scatter plots for all the variables
sns.set_style(style='darkgrid')
sns.pairplot(cars)



## **Preparing a model**

In [ ]:
#Build model
import statsmodels.formula.api as smf 
model=smf.ols('Price~Age_08_04+KM+HP+Doors+Gears+Weight',cars).fit()

In [ ]:
#coefficients
model.params

In [ ]:
#t and p values
print(model.tvalues,'\n',model.pvalues)

In [ ]:
#r squared values
(model.rsquared,model.rsquared_adj)

In [ ]:
rsq_Age_08_04 = smf.ols('HP~KM+HP+Doors+Gears+Weight',data=cars).fit().rsquared  
vif_Age_08_04 = 1/(1-rsq_Age_08_04) # 16.33

rsq_KM = smf.ols('KM~Age_08_04+HP+Doors+Gears+Weight',data=cars).fit().rsquared  
vif_KM = 1/(1-rsq_KM) # 564.98

rsq_HP = smf.ols('HP~Age_08_04+KM+Doors+Gears+Weight',data=cars).fit().rsquared  
vif_HP = 1/(1-rsq_HP) #  564.84

rsq_Doors = smf.ols('Doors~Age_08_04+KM+Gears+Weight+HP',data=cars).fit().rsquared  
vif_Doors = 1/(1-rsq_Doors) #  16.35

rsq_Gears = smf.ols('Gears~Age_08_04+HP+KM+Doors+Weight',data=cars).fit().rsquared
vif_Gears = 1/(1-rsq_Gears)

rsq_Weight = smf.ols('Weight~Age_08_04+HP+KM+Gears+Doors',data=cars).fit().rsquared
vif_Weight = 1/(1-rsq_Weight)

# Storing vif values in a data frame
d1 = {'Variables':['Age_08_04 ','KM','HP','Doors','Gears','Weight'],'VIF':[vif_Age_08_04 ,vif_KM,vif_HP,vif_Doors,vif_Gears,vif_Weight]}
Vif_frame = pd.DataFrame(d1)  
Vif_frame

# **Residual Analysis**

# **Test for normality of residuals**

In [ ]:
import statsmodels.api as sm
qqplot=sm.qqplot(model.resid,line='q') # line = 45 to draw the diagnoal line
plt.title("Normal Q-Q plot of residuals")
plt.show()

In [ ]:
list(np.where(model.resid>6000))

In [ ]:
list(np.where(model.resid<-7000))

# **Residual plot for homoscedascity**

In [ ]:
def get_standardized_values( vals ):
    return (vals - vals.mean())/vals.std()

In [ ]:
plt.scatter(get_standardized_values(model.fittedvalues),
            get_standardized_values(model.resid))

plt.title('Residual Plot')
plt.xlabel('Standardized Fitted values')
plt.ylabel('Standardized residual values')
plt.show()

# Resudual vs Regressors

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "Age_08_04", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "HP", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "KM", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "Doors", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "Weight", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "Gears", fig=fig)
plt.show()

# **Model Deletion Diagnostics**

# **Detecting influencers or outliers**

# **Cook's Distance**

In [ ]:
model_influence = model.get_influence()
(c, _) = model_influence.cooks_distance

In [ ]:
#Plot the influencers values using stem plot
fig = plt.subplots(figsize=(20, 7))
plt.stem(np.arange(len(cars)), np.round(c, 3))
plt.xlabel('Row index')
plt.ylabel('Cooks Distance')
plt.show()

In [ ]:
##index and value of influencer where c is more than .5
(np.argmax(c),np.max(c))

#High influence points

In [ ]:
from statsmodels.graphics.regressionplots import influence_plot
influence_plot(model)
plt.show()

In [ ]:
k = cars.shape[1]
n = cars.shape[0]
leverage_cutoff = 3*((k + 1)/n)

# **From the above plot,it is evident that data point is 221 and 960**

In [ ]:
cars[cars.index.isin([221,960])]

In [ ]:
#See the differences in HP and other variable values
cars.head()

In [ ]:
#Load the data
cars_new = pd.read_csv("Toyoto_Corrola.csv")

In [ ]:
#Discard the data points which are influencers and reasign the row number (reset_index())
car1=cars_new.drop(cars_new.index[[221,960]],axis=0).reset_index()

In [ ]:
car1

In [ ]:
#Exclude variable "WT" and generate R-Squared and AIC values
final_ml_P= smf.ols('Price~KM+HP+Weight+Age_08_04+Gears+Doors',data = cars).fit()

In [ ]:
(final_ml_P.rsquared,final_ml_P.aic,final_ml_P.bic)

Cook's Distance

In [ ]:
model_influence_P = final_ml_P.get_influence()
(c, _) = model_influence_P.cooks_distance

In [ ]:
fig= plt.subplots(figsize=(20,7))
plt.stem(np.arange(len(car1)),np.round(c,3));
plt.xlabel('Row index')
plt.ylabel('Cooks Distance')